# 1 - Install missing dependencies

For the plotting of the Land Cover map we need the earthpy python package, that relies on the libspatialindex-dev lib.

In [1]:
!apt-get -qq install -y libspatialindex-dev
!pip install -q --upgrade earthpy rasterio

Selecting previously unselected package libspatialindex4v5:amd64.
(Reading database ... 134448 files and directories currently installed.)
Preparing to unpack .../libspatialindex4v5_1.8.5-5_amd64.deb ...
Unpacking libspatialindex4v5:amd64 (1.8.5-5) ...
Selecting previously unselected package libspatialindex-c4v5:amd64.
Preparing to unpack .../libspatialindex-c4v5_1.8.5-5_amd64.deb ...
Unpacking libspatialindex-c4v5:amd64 (1.8.5-5) ...
Selecting previously unselected package libspatialindex-dev:amd64.
Preparing to unpack .../libspatialindex-dev_1.8.5-5_amd64.deb ...
Unpacking libspatialindex-dev:amd64 (1.8.5-5) ...
Setting up libspatialindex4v5:amd64 (1.8.5-5) ...
Setting up libspatialindex-c4v5:amd64 (1.8.5-5) ...
Setting up libspatialindex-dev:amd64 (1.8.5-5) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 1.4MB 2.8MB/s 
     

# 2 - Authenticate to Google Drive 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 3 - Import necessary python libraries and mount google drive

In [0]:
import os
import itertools

import numpy as np
import pandas as pd 
from scipy import stats
from scipy import optimize

import rasterio as rio
from rasterio.windows import Window

import matplotlib.pyplot as plt
import seaborn as sns
import earthpy.plot as ep
%matplotlib inline

# 4 - Read the data

We read all bands into a dictionary, that hold the band names as key and the respective array as value. All data is cut to the same extent of 3000 x 3000 pixels.


In [5]:
list_of_layers = [
    'VV_gamma0', 'VH_gamma0',
    'VV_gamma0flat', 'VH_gamma0flat',
    'alpha_rRad', 'theta_liaRad', 'aspect', 
    'layover', 'shadow',     
    'landcover']

# paths to dem and model types
dem = 'SRTMGL1_003'
models = ['volume', 'surface']
buffers = [0]

modelDict = {}

# loop thorugh all combinations and put into dictionary
for model, buffer in itertools.product(models, buffers):

    key = '{}_{}_buf_{}'.format(dem, model, buffer)
    # here we read all layers into a dictionary
    dataDict = {}
    for layer in list_of_layers:
        with rio.open('/content/drive/My Drive/slope_correction {}/{}.tif'.format(key, layer)) as src:
            print('Reading ' + layer)
            dataDict[layer] = np.nan_to_num(src.read(window=Window(0, 340, 3000, 3000)))[0]
            print(dataDict[layer].shape)
    
    # write respective dataDict to our model dict, where different models are stored
    modelDict[key] = dataDict

Reading VV_gamma0
(3000, 3000)
Reading VH_gamma0
(3000, 3000)
Reading VV_gamma0flat
(3000, 3000)
Reading VH_gamma0flat
(3000, 3000)
Reading alpha_rRad
(3000, 3000)
Reading theta_liaRad
(3000, 3000)
Reading aspect
(3000, 3000)
Reading layover
(3000, 3000)
Reading shadow
(3000, 3000)
Reading landcover
(3000, 3000)
Reading VV_gamma0
(3000, 3000)
Reading VH_gamma0
(3000, 3000)
Reading VV_gamma0flat
(3000, 3000)
Reading VH_gamma0flat
(3000, 3000)
Reading alpha_rRad
(3000, 3000)
Reading theta_liaRad
(3000, 3000)
Reading aspect
(3000, 3000)
Reading layover
(3000, 3000)
Reading shadow
(3000, 3000)
Reading landcover
(3000, 3000)


# 5 - Plotting functions

## 5.1 - plot aspect against backscatter function

In [0]:
def create_plot_aspect_against_backscatter(model, data, array, mask, stats_dict, outfile, gridsize):
    
    # getlayer info #
    polarisation = layer.split('_')[0]
    flat = layer.split('_')[1][-4:]
    if flat != 'flat':
        flat = False
    
    # fore and backslope lines
    look_angle = 82.30213964392819 # hardcoded, 
    backslope = look_angle
    foreslope = backslope + 180
    vertical_y = np.linspace(-28, 8, 5)
    fs_x = 0 * vertical_y + foreslope
    bs_x = 0 * vertical_y + backslope
    
    # calculate mean line
    horizontal_x = np.linspace(0, 360, 10) 
    mean_y = 0 * horizontal_x + stats_dict['mean']
    
    if not flat:
        y_label = r'$\gamma^0$ [dB]'
    else:
        y_label = r'$\gamma^0_f$ [dB]'
    
    # check for 0s in aspect and update mask
    data['aspect'][data['aspect'] == 0] = np.nan
    mask = mask & np.isfinite(data['aspect'])
    aspect_deg_masked = np.subtract(to_deg(data['aspect'][mask]), 180)
    aspect_deg_masked = to_deg(data['aspect'][mask])

    # plot
    # surpress plotting, since we only want to save the files
    plt.ioff()
    X = sns.jointplot(aspect_deg_masked, array[mask], kind='hex', gridsize=(gridsize, gridsize))
    X.ax_joint.plot(horizontal_x, mean_y, 'k--', linewidth=.75)
    X.ax_joint.plot(fs_x, vertical_y, 'k--', linewidth=.75)
    X.ax_joint.plot(bs_x, vertical_y, 'k--', linewidth=.75)
    X.ax_joint.set_xlabel(r'Aspect angle $\phi_s$ [deg]', fontsize=14)
    X.ax_joint.set_ylabel(y_label,  fontsize=14)
    X.ax_joint.set_ylim(-30, 10)
    #X.ax_joint.set_xlim(-190, 185)
    X.ax_joint.set_xlim(-10, 365)
    
    # add textbox with ampl, mean and sd
    props = dict(boxstyle='round', facecolor='lightgrey', alpha=0.5)
    textstr = '\n'.join((
        r'$\mathrm{A}=%.2f$' % (stats_dict['amplitude'], ),
        r'$\mu=%.2f$' % (stats_dict['mean'], ),
        r'$\sigma=%.2f$' % (stats_dict['sd'], )))
    X.ax_joint.text(290, -28,textstr, fontsize=10, bbox=props)
    
    # add title
    if not flat:
        title = '{} Backscatter modulation by slopes'.format(polarisation)
    else:
        title = '{} Backscatter after Model {}'.format(polarisation, model)
        
    plt.suptitle(title, x=0.45, y=1.02, fontweight='bold', fontsize=14)
    
    # save
    plt.savefig(outfile, bbox_inches='tight', pad_inches=0.5)
    plt.close()

## 5.2 - plot slope steepness against backscatter function

In [0]:
def create_plot_alpha_against_backscatter(model, data, array, mask, stats_dict, outfile, gridsize):
    
    # getlayer info #
    polarisation = layer.split('_')[0]
    flat = layer.split('_')[1][-4:]
    if flat != 'flat':
        flat = False
    
    # calculate mean line
    horizontal_x = np.linspace(-40, 40, 10) 
    mean_y = 0 * horizontal_x + stats_dict['mean']
    
    if not flat:
        y_label = r'$\gamma^0$ [dB]'
    else:
        y_label = r'$\gamma^0_f$ [dB]'
    
    # check for 0s in aspect and update mask
    data['alpha_rRad'][data['alpha_rRad'] == 0] = np.nan
    mask = mask & np.isfinite(data['alpha_rRad'])
    alpha_deg_masked = to_deg(data['alpha_rRad'][mask])

    # plot
    # surpress plotting, since we only want to save the files
    plt.ioff()
    X = sns.jointplot(alpha_deg_masked, array[mask], kind='hex', gridsize=(gridsize, gridsize))
    X.ax_joint.plot(horizontal_x, mean_y, 'k--', linewidth=.75)
    X.ax_joint.set_xlabel(r'Slope Steepness in range $\alpha$ [deg]', fontsize=14)
    X.ax_joint.set_ylabel(y_label, fontsize=14)
    X.ax_joint.set_ylim(-30, 10)
    X.ax_joint.set_xlim(-45, 45)
    
    # add textbox with ampl, mean and sd
    props = dict(boxstyle='round', facecolor='lightgrey', alpha=0.5)
    textstr = '\n'.join((
        r'$\mathrm{s}=%.2f$' % (stats_dict['slope'], ),
        r'$\mu=%.2f$' % (stats_dict['mean'], ),
        r'$\sigma=%.2f$' % (stats_dict['sd'], )))
    X.ax_joint.text(20, -28,textstr, fontsize=10, bbox=props)
    
    # add title
    if not flat:
        title = '{} Backscatter modulation by slopes'.format(polarisation)
    else:
        title = '{} Backscatter after Model {}'.format(polarisation, model)
        
    plt.suptitle(title, x=0.45, y=1.02, fontweight='bold', fontsize=14)
    
    # save
    plt.savefig(outfile, bbox_inches='tight', pad_inches=0.5)
    plt.close()

## 5.3 - plot lia against backscatter function

In [0]:
def create_plot_lia_against_backscatter(model, data, array, mask, stats_dict, outfile, gridsize):
    
    
    # getlayer info #
    polarisation = layer.split('_')[0]
    flat = layer.split('_')[1][-4:]
    if flat != 'flat':
        flat = False
    
    # calculate mean line
    horizontal_x = np.linspace(0, 90, 10) 
    mean_y = 0 * horizontal_x + stats_dict['mean']
    
    if not flat:
        y_label = r'$\gamma^0$ [dB]'
    else:
        y_label = r'$\gamma^0_f$ [dB]'
    
    # check for 0s in aspect and update mask
    data['theta_liaRad'][data['theta_liaRad'] == 0] = np.nan
    mask = mask & np.isfinite(data['theta_liaRad'])
    theta_deg_masked = to_deg(data['theta_liaRad'][mask])

    # plot
    # surpress plotting, since we only want to save the files
    plt.ioff()
    X = sns.jointplot(theta_deg_masked, array[mask], kind='hex', gridsize=(gridsize, gridsize))
    X.ax_joint.plot(horizontal_x, mean_y, 'k--', linewidth=.75)
    X.ax_joint.set_xlabel(r'Local Incidence Angle $\theta$ [deg]', fontsize=14)
    X.ax_joint.set_ylabel(y_label, fontsize=14)
    X.ax_joint.set_ylim(-30, 10)
    X.ax_joint.set_xlim(-10, 100)
    
    # add textbox with ampl, mean and sd
    props = dict(boxstyle='round', facecolor='lightgrey', alpha=0.5)
    textstr = '\n'.join((
        #r'$\mathrm{s}=%.2f$' % (stats_dict['slope'], ),
        r'$\mu=%.2f$' % (stats_dict['mean'], ),
        r'$\sigma=%.2f$' % (stats_dict['sd'], )))
    X.ax_joint.text(-5, -28,textstr, fontsize=10, bbox=props)
    
    # add title
    if not flat:
        title = '{} Backscatter modulation by slopes'.format(polarisation)
    else:
        title = '{} Backscatter after Model {}'.format(polarisation, model)
        
    plt.suptitle(title, x=0.45, y=1.02, fontweight='bold', fontsize=14)
    
    # save
    plt.savefig(outfile, bbox_inches='tight', pad_inches=0.5)
    plt.close()

# 6 - Statistics functions

In [0]:
# from rad to degree
def to_deg(rad):
    return np.multiply(rad,  np.divide(180,np.pi))

# sin function or curve fitting
def sin_func(x, a, b, c):
    return a * np.sin(x + b) - c

def tf_stats(data, array, layer, lc_class, mask):
    
    #------------------------------------------------
    # slope calculation
    
    # mask alpha angle
    data['alpha_rRad'][data['alpha_rRad'] == 0] = np.nan
    mask_alpha = mask & np.isfinite(data['alpha_rRad'])
    alpha_deg_masked = np.subtract(to_deg(data['alpha_rRad'][mask_alpha]), 180)
    
    # mask out nans
    x = array.copy()
    x[~mask_alpha] = np.nan
    x = x[np.logical_not(np.isnan(x))] 
    y = alpha_deg_masked
    y = y[np.logical_not(np.isnan(y))]
    
    # lin-regression
    slope, intercept, r_value, p_value, std_err = stats.linregress(y, x)        
    #------------------------------------------------
    
    #------------------------------------------------
    # amplitude calculation
    
    # mask aspect 0s and nans
    data['aspect'][data['aspect'] == 0] = np.nan
    mask_aspect = mask & np.isfinite(data['aspect'])
    aspect_deg_masked = np.subtract(data['aspect'][mask_aspect], np.pi)
    
    # mask out nans
    x = array.copy()
    x[~mask_aspect] = np.nan
    x = x[np.logical_not(np.isnan(x))]
    y = aspect_deg_masked
    y = y[np.logical_not(np.isnan(y))]
   
    # curve fitting
    params, params_covariance = optimize.curve_fit(sin_func, y, x)
    amp = params[0]
    #------------------------------------------------

    # mean, sd
    mean = np.nanmean(array[mask_alpha])
    std = np.nanstd(array[mask_alpha])
    
    # create final dictionary
    stat_dict = {'lc_class': lc_class, 
                 'layer': layer, 
                 'count': np.sum(mask), 
                 'mean': mean, 
                 'sd': std, 
                 'slope': slope, 
                 'amplitude': np.abs(amp)
                }
    
    return stat_dict

# 7 - Create figures and statistics dataframe or each model and land cover class

In [12]:
# class names
legend_entries = ["Build-up", "Flat sealed surfaces", "Permament soil", "Bare rock and screes", 
                  "Water", "Snow and ice", "Trees - broad-leaved", "Tree - coniferous", "Bushes and shrubs", "Herbacous periodically",
                  "Herbacous permanent - low productivity", "Herbacous permanent - high productivity", "Reeds"]
# class values
ras_values = [11, 12, 31, 32, 60, 70, 91, 93, 100, 122, 125, 126, 130]

# reduced list of class names
legend_entries = ["Bare rock and screes", "Trees - broad-leaved", 
                  "Tree - coniferous", "Bushes and shrubs", 
                  "Herbacous periodically", 
                  "Herbacous permanent - high productivity", "Reeds"]
# redued list of class values
ras_values = [32, 91, 93, 100, 122,  126, 130]

# prepare columns for new dataframe
df_cols = ['lc_class', 'layer', 'count', 'mean', 'sd', 'slope', 'amplitude']

# paths to dem and model types and buffer
dem = 'SRTMGL1_003'
models = ['surface', 'volume']
buffers = [0] 

# loop through different models and buffers
for model, buffer in itertools.product(models, buffers):
    
    # get respective arrays within the model/datadict
    key = '{}_{}_buf_{}'.format(dem, model, buffer)
    dataDict = modelDict[key]
    
    print(' INFO: Creating figures and stats for {} {} with buffer {}.'
      .format(dem, model, buffer)
    )
    # create empty dataframe for statistics
    df_stats = pd.DataFrame(columns=df_cols)
    
    # crate outdirectory where plots and stats will be saved
    outdir = '/content/drive/My Drive/slope_correction/pictures/{}/'.format(key)
    os.makedirs(outdir, exist_ok=True)
    
    # loop through classes and respective raster values file
    for legend_entry, ras_value in zip(legend_entries, ras_values):
        
        # set raster value respective to class
        print(' INFO: Analysing {} with raster value {}.'
          .format(legend_entry, ras_value)
        )

        # loop thorugh different corrected and uncorrected layers
        for layer in ['VV_gamma0', 'VV_gamma0flat', 'VH_gamma0', 'VH_gamma0flat']:
                        
            # create combined Land Cover and Layover/Shadow mask
            valid_data_mask = (
                [dataDict['landcover'] == ras_value] & 
                (dataDict['layover'] > 0) & 
                (dataDict['shadow'] > 0)
            )[0] 

            # apply this mask and add valid data mask of backscatter array 
            array = dataDict[layer].copy()
            array[array == 0] = np.nan
            mask = valid_data_mask & np.isfinite(array)

            # set everything else to nan
            array[~mask] = np.nan

            # for some classes array might be empty, so we add an if
            if True in np.unique(np.isfinite(array)):
                
                # calculate stats
                stats_dict = tf_stats(
                    dataDict.copy(), array, layer, legend_entry, mask
                )
                stats_dict['lc_class_code'] =  ras_value

                # and put into pandas dataframe
                df = pd.DataFrame([stats_dict], columns=stats_dict.keys())
                df_stats = df_stats.append(stats_dict, ignore_index=True)

                # plotting
                gridsize=100
                model_nr = '1' if model == 'volume' else '2'

                create_plot_aspect_against_backscatter(
                    model_nr, dataDict.copy(), array, mask, stats_dict, 
                    '{}/{}_{}_vs_aspect.png'.format(outdir, legend_entry, layer), 
                    gridsize
                )

                create_plot_alpha_against_backscatter(
                    model_nr, dataDict.copy(), array, mask, stats_dict, 
                    '{}/{}_{}_vs_slope.png'.format(outdir, legend_entry, layer), 
                    gridsize
                )

                create_plot_lia_against_backscatter(
                    model_nr, dataDict.copy(), array, mask, stats_dict, 
                    '{}/{}_{}_vs_LIA.png'.format(outdir, legend_entry, layer), 
                    gridsize
                )

        # save the complete stas dataframe to pickle
        df_stats.reset_index()
        df_stats.to_pickle('{}/stats.pickle'.format(outdir)) 

 INFO: Creating figures and stats for SRTMGL1_003 surface with buffer 0.
 INFO: Analysing Bare rock and screes with raster value 32.
 INFO: Analysing Trees - broad-leaved with raster value 91.
 INFO: Analysing Tree - coniferous with raster value 93.
 INFO: Analysing Bushes and shrubs with raster value 100.
 INFO: Analysing Herbacous periodically with raster value 122.
 INFO: Analysing Herbacous permanent - high productivity with raster value 126.
 INFO: Analysing Reeds with raster value 130.
 INFO: Creating figures and stats for SRTMGL1_003 volume with buffer 0.
 INFO: Analysing Bare rock and screes with raster value 32.
 INFO: Analysing Trees - broad-leaved with raster value 91.
 INFO: Analysing Tree - coniferous with raster value 93.
 INFO: Analysing Bushes and shrubs with raster value 100.
 INFO: Analysing Herbacous periodically with raster value 122.
 INFO: Analysing Herbacous permanent - high productivity with raster value 126.
 INFO: Analysing Reeds with raster value 130.


# 8 - re-organise statistics dataframe for creation of Table 1

In [18]:
df_stats_dict = {}

# paths to dem and model types
dem = 'SRTMGL1_003'
models = ['surface', 'volume'] 

# this is for the concatenation of VV and VH stats
concat_cols = ['lc_class', 'lc class code', 'VV', 'Pixel count', 'VV mean', 'VV SD', 'VV slope', 'VV amp', 
                'VH', 'VH mean', 'VH SD', 'VH slope', 'VH amp']

# create empty list for adding all stats within the for loop
df_con_merged = []

# loop through different dems and models
for model in models:

    # get respective arrays within the datadict
    key = '{}_{}_buf_0'.format(dem, model)
    
    # read each df into the dictionary
    outdir = '/content/drive/My Drive/slope_correction/pictures/{}/'.format(key)
    df_stats = pd.read_pickle('{}/stats.pickle'.format(outdir))
    
    # split into vv and vh
    df_vv = df_stats[df_stats['layer'].str.contains('VV')].reset_index().rename(columns={'layer': 'VV-pol'})
    df_vh = df_stats[df_stats['layer'].str.contains('VH')].reset_index().rename(columns={'layer': 'VH-pol'})
    
    # concat vv and vh columns
    df_con = pd.concat([df_vv, df_vh], axis=1, ignore_index=True)
    
    # rename columns
    df_con.columns = ['i_2', 'lc_class', 'VV', 'Pixel count', 'VV mean', 'VV SD', 'VV slope', 'VV amp', 'lc class code',
                      'i_3', 'lc_class_2', 'VH', 'VH count', 'VH mean', 'VH SD', 'VH slope', 'VH amp', 'lc class code_2']
    
    # subset columns
    df_con = df_con[['lc_class', 'lc class code', 'VV', 'Pixel count', 'VV mean', 'VV SD', 'VV slope', 'VV amp', 
                     'VH', 'VH mean', 'VH SD', 'VH slope', 'VH amp']]
    
    # rename the layer names 
    df_con['VV'] = df_con['VV'].str.replace('VV_gamma0flat', '{}'.format(key))
    df_con['VH'] = df_con['VH'].str.replace('VH_gamma0flat', '{}'.format(key))
    
    df_con['VV'] = df_con['VV'].str.replace('VV_gamma0', 'Original')
    df_con['VH'] = df_con['VH'].str.replace('VH_gamma0', 'Original')
    
    # merge to existent dfs
    df_con_merged.append(df_con)


# bring all the data together
appended_data = pd.concat(df_con_merged)

# exclude marginal classes
appended_data = appended_data[appended_data['Pixel count'] >= 100000]
appended_data = appended_data[['lc_class', 'VV', 'VV mean', 'VV SD', 'VV slope', 'VV amp', 'VH mean', 'VH SD', 'VH slope', 'VH amp']]

# rename model names
appended_data['VV'] = appended_data['VV'].str.replace('SRTMGL1_003_volume_buf_0', 'Model I')
appended_data['VV'] = appended_data['VV'].str.replace('SRTMGL1_003_surface_buf_0', 'Model II')

# remove double entries for original data and reindex 
appended_data.drop_duplicates(subset=['lc_class', 'VV'], keep='first', inplace=True)
appended_data = appended_data.set_index(['lc_class', 'VV']).sort_values(['lc_class', 'VV'], ascending=False)

# rename model column
appended_data

VV mean  ...    VH amp
lc_class                                VV                   ...          
Trees - broad-leaved                    Original  -8.742994  ...  3.743469
                                        Model II  -8.957283  ...  1.807970
                                        Model I   -7.897023  ...  1.302934
Tree - coniferous                       Original  -7.840942  ...  4.311772
                                        Model II  -8.678100  ...  1.902061
                                        Model I   -7.885640  ...  1.392571
Herbacous permanent - high productivity Original -10.386123  ...  2.236016
                                        Model II -10.667173  ...  0.915310
                                        Model I  -10.147956  ...  1.102624
Herbacous periodically                  Original  -8.742648  ...  0.757547
                                        Model II  -8.767311  ...  0.289349
                                        Model I   -8.621692  ...  0.487819
Bushes and shrubs                       Original  -9.406825  ...  4.273665
                                        Model II -10.131575  ...  1.812582
                                        Model I   -8.815239  ...  1.648927
Bare rock and screes                    Original  -7.993525  ...  5.180944
                                        Model II  -8.512291  ...  2.578791
                                        Model I   -6.749642  ...  1.685885

[18 rows x 8 columns]

# 9 - Export statistics as latex table 

In [0]:
with open('/content/drive/My Drive/slope_correction/table_all.tex', 'w') as tf:
     tf.write(appended_data.to_latex(columns = ['VV mean', 'VV SD', 'VV slope', 'VV amp', 'VH mean', 'VH SD', 'VH slope', 'VH amp'], index=True, bold_rows=True, escape=True, float_format=lambda x: '%10.3f' % x))